# Import Libraries

In [ ]:
import os
import glob
import matplotlib.pyplot as plt
from PIL import Image
#
import tensorflow as tf
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Flatten, GlobalAveragePooling2D,Conv2D, MaxPooling2D,BatchNormalization, Dropout, Input, LeakyReLU
from tensorflow.keras.applications.resnet50 import ResNet50, preprocess_input
from tensorflow.keras.applications import VGG16
from tensorflow.keras.models import Model


# Kaggle API Token

1. Go to your Kaggle account, click on your profile picture (top right), and go to "Account" from the menu.
2. Scroll down to the "API" section and click on "Create New API Token". This will download a kaggle.json file containing your API credentials.

In [ ]:
from google.colab import files
files.upload()

# Setup Kaggle API
Set up the Kaggle API and ensure the kaggle.json file is in the correct location with the appropriate permissions.

In [ ]:
!pip install kaggle --quiet
!mkdir -p ~/.kaggle
!cp kaggle.json ~/.kaggle/
!chmod 600 ~/.kaggle/kaggle.json

# Download the Dataset

In [ ]:
#kaggle datasets download -d [username/dataset-name]
!kaggle datasets download -d paultimothymooney/chest-xray-pneumonia

 # Extract the Dataset

In [ ]:
!unzip -q chest-xray-pneumonia.zip

# List Directories at the Root of the Unzipped Folder
Assuming the dataset is unzipped into a folder named chest_xray in the current working directory, you can list its contents using os.listdir().

In [ ]:
root_dir = 'chest_xray'
os.listdir(root_dir)

# Explore Further and List Files
To list the files in a specific directory, such as the training set for normal chest X-rays, you might do something like this:

In [ ]:
train_normal_dir = os.path.join(root_dir, 'train', 'NORMAL')
train_normal_files = os.listdir(train_normal_dir)
print(train_normal_files[:4])  # List the first 10 file names


In [ ]:
train_pneumonia_dir = os.path.join(root_dir, 'train', 'PNEUMONIA')
train_pneumonia_files = glob.glob(train_pneumonia_dir + '/**/*.jpeg', recursive=True)
print(train_pneumonia_files[:10])  # List the first 10 file paths

# Display Multiple Images in a Grid
To display multiple images in a grid, you can use a loop. Here's an example of displaying the first few images from a list of image paths:

In [ ]:
# Example: Display the first 3 pneumonia images from the training set
num_images = 3
fig, axes = plt.subplots(1, 3, figsize=(12, 12))  # Create a grid of 3x3

for i, ax in enumerate(axes.flat):
    if i < num_images:
        image_path = train_pneumonia_files[i]
        image = Image.open(image_path)
        ax.imshow(image, cmap='gray')  # Use cmap='gray' if it's a grayscale image
        ax.set_title(f'Image {i+1}')
        ax.axis('off')  # Hide the axes
    else:
        ax.axis('off')  # Hide axes for empty subplots

plt.tight_layout()
plt.show()


# Data Preprocessing
Use ImageDataGenerator to load your data and apply preprocessing required for ResNet50. You might need to adjust target_size based on the input size that ResNet50 expects (224x224). You can change the images to grayscale as needed.

In [ ]:
train_dir = os.path.join(root_dir, 'train')
val_dir = os.path.join(root_dir, 'val')
test_dir = os.path.join(root_dir, 'test')

In [ ]:
# Move 24 images from the training set to the validation set
num_images = 24

normal_images = train_normal_files[:num_images]
pneumonia_images = train_pneumonia_files[:num_images]

for image in normal_images:
      source = os.path.join(train_normal_dir, image)
      dest = os.path.join(val_dir, 'NORMAL', image)
      os.rename(source, dest)

for image in pneumonia_images:
      source = image
      dest = os.path.join(val_dir, 'PNEUMONIA', os.path.basename(image))
      os.rename(source, dest)

# Check the number of images in the training and validation sets
num_train_normal = len(os.listdir(train_normal_dir))
num_train_pneumonia = len(os.listdir(train_pneumonia_dir))

num_val_normal = len(os.listdir(os.path.join(val_dir, 'NORMAL')))
num_val_pneumonia = len(os.listdir(os.path.join(val_dir, 'PNEUMONIA')))
print(f'Training set: Normal - {num_train_normal}, Pneumonia - {num_train_pneumonia}')
print(f'Validation set: Normal - {num_val_normal}, Pneumonia - {num_val_pneumonia}')

                        

In [ ]:

# Define the preprocessing function
def preprocess(image, label):
    image = preprocess_input(image)
    # apply random brightness augmentation
    image = tf.image.random_brightness(image, 0.1)
  # apply random contrast augmentation
    image = tf.image.random_contrast(image, 0.4, 0.6)
    
    return image, label

# Create a dataset from the directory
train_dataset = tf.keras.preprocessing.image_dataset_from_directory(
    train_dir,
    image_size=(224, 224),
    batch_size=32,
    label_mode='binary',
    preprocessing_function=preprocess
)

# Apply preprocessing, augmentation and prefetching
train_dataset = train_dataset.map(preprocess, num_parallel_calls=tf.data.AUTOTUNE)
train_dataset = train_dataset.prefetch(tf.data.AUTOTUNE)

# Do the same for the validation dataset
val_dataset = tf.keras.preprocessing.image_dataset_from_directory(
    val_dir,
    image_size=(224, 224),
    batch_size=32,
    label_mode='binary'
)
val_dataset = val_dataset.map(preprocess, num_parallel_calls=tf.data.AUTOTUNE)
val_dataset = val_dataset.prefetch(tf.data.AUTOTUNE)

all_data = tf.keras.preprocessing.image_dataset_from_directory(
    root_dir,
    image_size=(224, 224),
    batch_size=32,
    label_mode='binary'
)
all_data = all_data.map(preprocess, num_parallel_calls=tf.data.AUTOTUNE)
all_data = all_data.prefetch(tf.data.AUTOTUNE)



# Load VGG16 trained on ImageNet

In [ ]:
# Load pre-trained VGG16 VGG_custom_head without head (headless)
vgg_ = VGG16(
    weights='imagenet',
    include_top=False,
    input_shape=(224, 224, 3)
)
#vgg_.trainable = False
vgg_.trainable = True

# Define input layer (adjust the shape based on your input images)
input_tensor = Input(shape=(224, 224, 3), name='input_1')

# Pass input through VGG16 base VGG_custom_head
x = vgg_(input_tensor, training=False)
x = BatchNormalization()(x)
x = Conv2D(128, (3, 3), activation='relu', name='Conv2D_1')(x)
x = Flatten()(x)
x = Dense(256, activation='relu', name='Dense_1')(x)
x = Dropout(0.5, name='Dropout_1')(x)
x = BatchNormalization()(x)
VGG_output = Dense(1, activation='sigmoid', name='Output_1')(x)

# Create the VGG_custom_head model
VGG_custom_head = Model(inputs=input_tensor, outputs = VGG_output, name='VGG16_transfer')

base_learning_rate = 0.0001

# Fine-tune layers the last 4 layers
number_of_tune_layers = 2
for layer in vgg_.layers[:-number_of_tune_layers]:
    layer.trainable = False

# Compile the VGG_custom_head
VGG_custom_head.compile(
    optimizer=tf.keras.optimizers.Adam(learning_rate=base_learning_rate),
    loss=tf.keras.losses.BinaryCrossentropy(),
    metrics=['accuracy']
)

# Print VGG_custom_head and the base summary
vgg_.summary()
VGG_custom_head.summary()
tf.keras.utils.plot_model(VGG_custom_head, show_shapes=True)

# Define callbacks
early_stopping = tf.keras.callbacks.EarlyStopping(
    monitor='val_accuracy',
    patience=3,
    restore_best_weights=True
)


In [ ]:
# Train the model
VGG_history = VGG_custom_head.fit(
    train_dataset,
    validation_data=val_dataset,
    epochs=10,
    callbacks=[early_stopping],
    verbose=2
)

# Load the Pre-trained ResNet50 Model
Load ResNet50 with pre-trained weights from ImageNet, excluding the top (fully connected) layers. Set input_shape as required by ResNet50.

In [ ]:
resnet50_ = ResNet50(weights='imagenet', include_top=False, input_shape=(224, 224, 3))
# Fine-tune layers the last 4 layers
number_of_tune_layers = 18
for layer in resnet50_.layers[:-number_of_tune_layers]:
    layer.trainable = False

# Add Custom Layers on Top of ResNet50
Now, create a sequential model, add the pre-trained base model, and then add your custom layers on top.

In [ ]:
# Add your sequential model 

# Create a new model on top
Res_input = Input(shape=(224, 224, 3))
x = resnet50_(Res_input, training=False)
x = BatchNormalization()(x)
x = Conv2D(128, (5, 5), activation='relu')(x)
x = MaxPooling2D((2, 2))(x)
x = Dropout(0.5)(x)
x = Flatten()(x)
x = Dense(128, activation='relu')(x)
x = Dropout(0.5)(x)
Res_outputs = Dense(1, activation='sigmoid')(x)

Res_model = Model(Res_input, Res_outputs)
base_learning_rate = 0.0001

# Compile the model
Res_model.compile(
      optimizer=tf.keras.optimizers.Adam(learning_rate=base_learning_rate),
      loss=tf.keras.losses.BinaryCrossentropy(),
      metrics=['accuracy']
   )

resnet50_.summary()
Res_model.summary()

#tf.keras.utils.plot_model(Res_model, show_shapes=True)

In [ ]:
# Fit your model
Res_history = Res_model.fit(
    train_dataset,
    validation_data=val_dataset,
    epochs=30,
    callbacks=[early_stopping],
)

# Prepare the Test Data
First, prepare your test data using ImageDataGenerator, similar to how you prepared the training and validation data. Make sure to use the preprocess_input function from ResNet50 for preprocessing.

In [ ]:
# Add your code by following the example in the Data Preprocessing section.
test_dataset = tf.keras.preprocessing.image_dataset_from_directory(
    test_dir,
    image_size=(224, 224),
    batch_size=32,
    label_mode='binary'
)
test_dataset = test_dataset.map(preprocess, num_parallel_calls=tf.data.AUTOTUNE)
test_dataset = test_dataset.prefetch(tf.data.AUTOTUNE)



# Evaluate the Model
Use the evaluate method to test the model's performance on the test set. Your model must have at least 90% accuracy.

In [ ]:
# Evaluate the model
VGG_custom_head.evaluate(test_dataset, verbose=1)
Res_model.evaluate(test_dataset, verbose=1)

# Plot Learning Curves
To visualize the learning curves (accuracy and loss for both training and validation), you can plot them using matplotlib. This helps in understanding overfitting, underfitting, or if the model is learning well.

In [ ]:
# Add your code here or use tensorboard
import plotly.graph_objects as go

# Create traces
trace1 = go.Scatter(
   x=list(range(len(VGG_history.history['accuracy']))),
   y=VGG_history.history['accuracy'],
   mode='lines',
   name='VGG16 Training Accuracy'
)

trace2 = go.Scatter(
   x=list(range(len(VGG_history.history['val_accuracy']))),
   y=VGG_history.history['val_accuracy'],
   mode='lines',
   name='VGG16 Validation Accuracy'
)

trace3 = go.Scatter(
   x=list(range(len(Res_history.history['accuracy']))),
   y=Res_history.history['accuracy'],
   mode='lines',
   name='ResNet50 Training Accuracy'
)

trace4 = go.Scatter(
   x=list(range(len(Res_history.history['val_accuracy']))),
   y=Res_history.history['val_accuracy'],
   mode='lines',
   name='ResNet50 Validation Accuracy'
)

data = [trace1, trace2, trace3, trace4]

# Define layout
layout = go.Layout(
   title='Training and Validation Accuracy for VGG16 and ResNet50',
   xaxis=dict(title='Epoch'),
   yaxis=dict(title='Accuracy'),
)

# Define figure
fig = go.Figure(data=data, layout=layout)

# Plot the figure
fig.show()


# Create a Customized Model and Evaluate It.
 Your customized model must have at least 90% accuracy.


In [ ]:
# Add your code here

input_tensor = Input(shape=(224, 224, 3), name='input_01')

x = Conv2D(8, (3, 3), activation='relu', name='block_1_conv_1')(input_tensor)
x = BatchNormalization(name='block_1_batchNorm')(x)
x = MaxPooling2D((2, 2), name='block_1_maxPooling')(x)
x = Dropout(0.0, name='block_1_dropout')(x)

x = Conv2D(16, (5, 5), activation='relu', name='block_2_conv_1')(x)
x = BatchNormalization(name='block_2_batchNorm')(x)
x = MaxPooling2D((2, 2), name='block_2_maxPooling')(x)
x = Dropout(0.1, name='block_2_dropout')(x)

x = Conv2D(32, (5, 5), activation='relu', name='block_3_conv_1')(x)
x = BatchNormalization(name='block_3_batchNorm')(x)
x = MaxPooling2D((2, 2), name='block_3_maxPooling')(x)
x = Dropout(0.5, name='block_3_dropout')(x)

x = Conv2D(128, (7, 7), activation='relu', name='block_4_conv_1')(x)
x = BatchNormalization(name='block_4_batchNorm_1')(x)
x = Conv2D(128, (7, 7), activation='relu', name='block_4_conv_2')(x)
x = BatchNormalization(name='block_4_batchNorm_2')(x)
x = Conv2D(128, (7, 7), activation='relu', name='block_4_conv_3')(x)
x = BatchNormalization(name='block_4_batchNorm_3')(x)
x = MaxPooling2D((2, 2), name='block_4_maxPooling')(x)
x = Dropout(0.5, name='block_4_dropout')(x)

x = Flatten(name='Flatten')(x)
#x = GlobalAveragePooling2D()(x)

x = Dense(128, activation='relu',name='fc_1')(x)
x = BatchNormalization(name='fc_batchNorm')(x)
x = Dropout(0.5, name='block_5_dropout')(x)

custom_output = Dense(1, activation='sigmoid',name='output_1')(x)
custom_model = Model(inputs=input_tensor, outputs=custom_output, name='custom_model')

base_learning_rate = 0.001

# Compile the model
custom_model.compile(
    optimizer=tf.keras.optimizers.SGD(learning_rate=base_learning_rate),
    loss=tf.keras.losses.BinaryCrossentropy(),
    metrics=['accuracy']
)

custom_model.summary()

# Fit your model
custom_history = custom_model.fit(
    train_dataset,
    validation_data=val_dataset,
    epochs=30,
    callbacks=[early_stopping],
)

# Evaluate the model
custom_model.evaluate(test_dataset, verbose=1)


In [ ]:
# Add your code here

input_tensor = Input(shape=(224, 224, 3), name='input_01')

x = Conv2D(32, (5, 5), activation='relu', name='block_1_conv_1')(input_tensor)
x = BatchNormalization(name='block_1_batchNorm_1')(x)
x = Conv2D(32, (5, 5), activation='relu', name='block_1_conv_2')(x)
x = BatchNormalization(name='block_1_batchNorm_2')(x)
x = Conv2D(32, (5, 5), activation='relu', name='block_1_conv_3')(x)
x = BatchNormalization(name='block_1_batchNorm_3')(x)
x = MaxPooling2D((2, 2), name='block_1_maxPooling')(x)
x = Dropout(0.5, name='block_1_dropout')(x)

x = Conv2D(64, (7, 7), activation='relu', name='block_2_conv_1')(x)
x = BatchNormalization(name='block_2_batchNorm_1')(x)
x = Conv2D(64, (7, 7), activation='relu', name='block_2_conv_2')(x)
x = BatchNormalization(name='block_2_batchNorm_2')(x)
x = Conv2D(64, (7, 7), activation='relu', name='block_2_conv_3')(x)
x = BatchNormalization(name='block_2_batchNorm_3')(x)
x = MaxPooling2D((2, 2), name='block_2_maxPooling')(x)
x = Dropout(0.5, name='block_2_dropout')(x)

x = Conv2D(128, (7, 7), activation='relu', name='block_4_conv_1')(x)
x = BatchNormalization(name='block_4_batchNorm_1')(x)
x = Conv2D(128, (7, 7), activation='relu', name='block_4_conv_2')(x)
x = BatchNormalization(name='block_4_batchNorm_2')(x)
x = Conv2D(128, (7, 7), activation='relu', name='block_4_conv_3')(x)
x = BatchNormalization(name='block_4_batchNorm_3')(x)
x = MaxPooling2D((2, 2), name='block_4_maxPooling')(x)
x = Dropout(0.5, name='block_4_dropout')(x)

x = Flatten(name='Flatten')(x)
#x = GlobalAveragePooling2D()(x)

x = Dense(128, activation='relu',name='fc_1')(x)
x = BatchNormalization(name='fc_batchNorm')(x)
x = Dropout(0.5, name='block_5_dropout')(x)

custom_output = Dense(1, activation='sigmoid',name='output_1')(x)
custom_model = Model(inputs=input_tensor, outputs=custom_output, name='custom_model')

base_learning_rate = 0.001

# Compile the model
custom_model.compile(
    optimizer=tf.keras.optimizers.Adam(learning_rate=base_learning_rate),
    loss=tf.keras.losses.BinaryCrossentropy(),
    metrics=['accuracy']
)

custom_model.summary()

# Fit your model
custom_history = custom_model.fit(
    train_dataset,
    validation_data=val_dataset,
    epochs=30,
    callbacks=[early_stopping],
)

# Evaluate the model
custom_model.evaluate(test_dataset, verbose=1)


In [ ]:
# Use learning curves or tensorboard to visualize the performance of your customized model.
import plotly.graph_objects as go

# Create traces
trace1 = go.Scatter(
   x=list(range(len(custom_history.history['accuracy']))),
   y=custom_history.history['accuracy'],
   mode='lines',
   name='Custom Model Training Accuracy'
)

trace2 = go.Scatter(
   x=list(range(len(custom_history.history['val_accuracy']))),
   y=custom_history.history['val_accuracy'],
   mode='lines',
   name='Custom Model Validation Accuracy'
)

trace3 = go.Scatter(
   x=list(range(len(custom_history.history['loss']))),
   y=custom_history.history['loss'],
   mode='lines',
   name='Custom Model Training Loss'
)

trace4 = go.Scatter(
   x=list(range(len(custom_history.history['val_loss']))),
   y=custom_history.history['val_loss'],
   mode='lines',
   name='Custom Model Validation Loss'
)

data = [trace1, trace2, trace3, trace4]

# Define layout
layout = go.Layout(
   title='Training and Validation Accuracy for Custom Model',
   xaxis=dict(title='Epoch'),
   yaxis=dict(title='Accuracy'),
)

# Define figure
fig = go.Figure(data=data, layout=layout)

# Plot the figure
fig.show()



In [ ]:
# Submit a separate Word or PDF document discussing the performances of the transfer learning model vs. your customized model.